# Hugging Face Transformers

1. Text Classification
You can use the pipeline function for quick experimentation:

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 32.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.5/381.5 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 33.9 MB/s eta 0:00:0000:0100:01


In [15]:
pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 54.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.65.0
    Uninstalling tqdm-4.65.0:
      Successfully uninstalled tqdm-4.65.0
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:

In [7]:
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model="distilbert/distilbert-base-uncased-finetuned-sst-2-english",
    framework="pt",
    device=0  # explicitly specify PyTorch (though this is default)
)
result = classifier("I love using Hugging Face Transformers!")
print(result)


[{'label': 'POSITIVE', 'score': 0.9971315860748291}]


This will give you a quick result on the sentiment of the text.

To fine-tune a text classification model:


In [22]:
!pip uninstall pyarrow
# pip install pyarrow==12.0.1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: pyarrow 12.0.1
Uninstalling pyarrow-12.0.1:
  Would remove:
    /opt/anaconda3/lib/python3.11/site-packages/pyarrow-12.0.1.dist-info/*
    /opt/anaconda3/lib/python3.11/site-packages/pyarrow/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [23]:
# First, try just the dataset import
from datasets import load_dataset
dataset = load_dataset("imdb")
print("Dataset loaded successfully")

# Then try tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
print("Tokenizer loaded successfully")

# Then try model
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")
print("Model loaded successfully")

AttributeError: module 'pyarrow.lib' has no attribute 'ListViewType'

In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch


# Load dataset and model
dataset = load_dataset("imdb")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2  # binary classification for IMDB
)

# Set device
device = "mps" if torch.backends.mps.is_available() else "cpu"
model = model.to(device)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(
        examples["text"], 
        truncation=True, 
        padding=True,
        return_tensors="pt"  # explicitly request PyTorch tensors
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Fine-tuning
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
    no_cuda=True,  # disable CUDA
    use_mps_device=True if device == "mps" else False  # enable MPS if available
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"]
)

trainer.train()

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

2. Named Entity Recognition (NER)
For Named Entity Recognition, you can use:

In [11]:
from transformers import pipeline

ner = pipeline("ner", grouped_entities=True)
result = ner("Hugging Face Inc. is a company based in New York City.")
print(result)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

RuntimeError: Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")


3. Text Generation
To generate text using GPT-2:



In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="gpt2")
result = generator("Once upon a time", max_length=50, num_return_sequences=1)
print(result)

# If you want to fine-tune GPT-2, you can use:

from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


4. Text Summarization
For text summarization, use BART or T5:

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization")
text = """
Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, therefore very close to the Manhattan Bridge.
Hugging Face is known for its open-source library Transformers, which provides pre-trained models for natural language processing.
"""
result = summarizer(text, max_length=50, min_length=25, do_sample=False)
print(result)

5. Question Answering
For question answering, you can do:

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering")
result = question_answerer(question="Where is Hugging Face based?", context="Hugging Face Inc. is based in New York City.")
print(result)

6. Translation Example
For translation, you can use the mBART model:

In [ ]:
from transformers import pipeline

translator = pipeline("translation_en_to_fr", model="Helsinki-NLP/opus-mt-en-fr")
result = translator("Hello, how are you?")
print(result)

7. 迁移学习 (Transfer Learning)

迁移学习是利用在大型数据集上训练的预训练模型（如 BERT、GPT-2），然后对特定任务进行微调。示例代码如上所示，BERT 微调情感分析任务即为迁移学习的典型应用。


8. 零样本学习 (Zero-Shot Learning)

使用零样本分类模型（例如 BART 或 RoBERTa）进行分类：

In [ ]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
sequence_to_classify = "This is a great product, I love it!"
candidate_labels = ["positive", "negative", "neutral"]
result = classifier(sequence_to_classify, candidate_labels)
print(result)

9. 少量样本学习 (Few-Shot Learning)

利用 GPT-3 或类似的大型语言模型，通过提供几个示例进行少量样本学习：

In [ ]:
import openai

openai.api_key = 'your-api-key'

prompt = """
The following are examples of classifying movie reviews as positive or negative:

Review: "This movie was amazing, the acting was great!"
Label: Positive

Review: "The film was dull and boring."
Label: Negative

Review: "I had a great time watching it."
Label:
"""
response = openai.Completion.create(
    engine="text-davinci-003",
    prompt=prompt,
    max_tokens=5
)

print(response.choices[0].text.strip())

10. 从0训练模型 (Training from Scratch)

从头开始训练一个 Transformer 模型：

In [ ]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# 定义模型配置
config = AutoConfig.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification(config)

# 加载数据集
dataset = load_dataset("imdb")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 对数据进行tokenize
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 定义训练参数
training_args = TrainingArguments(output_dir="./results", evaluation_strategy="epoch")

# 使用 Trainer 进行训练
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"]
)

trainer.train()

有难度的类型

除了上面提到的常见任务，NLP 中还有一些更具挑战性的任务：

多任务学习 (Multi-Task Learning)：训练一个模型同时完成多个不同类型的任务，例如同时进行文本分类和命名实体识别。

领域自适应 (Domain Adaptation)：将模型从一个领域迁移到另一个领域，通常需要应对不同领域之间的差异，例如从新闻数据迁移到医学数据。

强化学习 (Reinforcement Learning for NLP)：在对话生成或摘要中使用强化学习来优化生成质量。

元学习 (Meta Learning)：通过学习如何学习，提高模型在少样本场景中的表现。

对抗性训练 (Adversarial Training)：训练模型在面对对抗性输入时具有更好的鲁棒性，防止模型受到对抗性攻击的影响。